<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/main/15_optimization/040_Global_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy.optimize as so
import matplotlib.pyplot as plt



# 전역최적화 사례 Global Optimization Examples



여러 국소 최소점을 가진 비용 함수를 생각해 보자.<br>Let's think about a cost function with multiple local minima.



In [ ]:
def a_func_not_smooth(x):
    return np.sin(x*10) + 0.25 * x * x



In [ ]:
def df_dx_cost(x):
    return 10.0 * np.cos(x * 10.0) + 0.5 * x



In [ ]:
def plot_the_func_not_smooth():
    x_array = np.linspace(-1, 1, 200)
    y_array = a_func_not_smooth(x_array)
    plt.plot(x_array, y_array)
    plt.grid(True)


plot_the_func_not_smooth()
plt.show()
plt.close()



## 국소 최적화 알고리듬의 경우<br>A case of a Local Optimization Algorithm



국소 최적화 알고리듬은 "충분히 가깝지 않은" 경우 전역 최적점을 찾기 힘들 수도 있다.<br>A local optimization might have challenges finding the global minimum.



In [ ]:
not_the_best_initial_guess = -0.7
result_not_the_best_initial_guess = so.minimize(
    a_func_not_smooth, not_the_best_initial_guess,
    jac=df_dx_cost,
)

plot_the_func_not_smooth()
plt.plot(result_not_the_best_initial_guess.x, a_func_not_smooth(result_not_the_best_initial_guess.x), 'o')
plt.show()
plt.close()



## 전역 최적화 알고리듬의 경우<br>Cases of Global Optimization Algorithms



중간 과정의 그래프를 그려 주는 비용 함수를 선언<br>Declare another cost function that will plot intermediate results



In [ ]:
class RoughCostFunctionLogger():
    def __init__(self):

        self.x_log = []
        self.y_log = []

        self.x_plot = np.linspace(-10, 10, 201)
        self.y_plot = a_func_not_smooth(self.x_plot)

    def plot_cost_function(self):
        plt.plot(self.x_plot, self.y_plot)
        plt.grid(True)

    def cost(self, x:np.ndarray) -> float:
        result = a_func_not_smooth(x)

        self.x_log.append(x[0])
        self.y_log.append(result)

        return result

    def plot_cost_function_and_log(self):
        self.plot_cost_function()
        plt.scatter(self.x_log, self.y_log, c=range(len(self.x_log)))
        plt.colorbar()



### Basin Hopping



In [ ]:
logger_basinhopping = RoughCostFunctionLogger()



In [ ]:
%%time
result_basinhopping = so.basinhopping(logger_basinhopping.cost, [-0.7])
result_basinhopping



In [ ]:
logger_basinhopping.plot_cost_function_and_log()
plt.axvline(result_basinhopping.x, color="red");
plt.show()
plt.close()



### Brute



In [ ]:
logger_brute = RoughCostFunctionLogger()



In [ ]:
%%time
result_brute = so.brute(logger_brute.cost, ((-5.0, 5.0),))
result_brute, a_func_not_smooth(result_brute)



In [ ]:
logger_brute.plot_cost_function_and_log()
plt.axvline(result_brute, color="red");
plt.show()
plt.close()



### Differential Evolution



In [ ]:
logger_de = RoughCostFunctionLogger()



In [ ]:
%%time
result_de = so.differential_evolution(logger_de.cost, ((-6.0, 6.0),))
result_de



In [ ]:
logger_de.plot_cost_function_and_log()
plt.axvline(result_de.x, color="red");
plt.show()
plt.close()



### SHGO



In [ ]:
logger_shgo = RoughCostFunctionLogger()



In [ ]:
%%time
result_shgo = so.shgo(logger_shgo.cost, ((-6.0, 6.0),))
result_shgo



In [ ]:
logger_shgo.plot_cost_function_and_log()
plt.axvline(result_shgo.x, color="red");
plt.show()
plt.close()



### Dual annealing



In [ ]:
logger_da = RoughCostFunctionLogger()



In [ ]:
%%time
result_da = so.dual_annealing(logger_da.cost, ((-6.0, 6.0),))
result_da



In [ ]:
logger_da.plot_cost_function_and_log()
plt.axvline(result_da.x, color="red");
plt.show()
plt.close()



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

